In [1]:
import pandas as pd
import numpy as np
import pickle, logging, spacy, sys, os, json, requests
import matplotlib.pyplot as plt

from helpers.classes import Collection
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

from helpers.cloze_generation import \
    generate_clozes_from_point, named_entity_answer_generator as ne_answer_generator, noun_phrase_answer_generator as np_answer_generator

from helpers.language_modelling import \
    find_nth_substring, multitoken_prediction, check_model, run_language_model

In [2]:
DATASETS = os.listdir('datasets')

with open('pickles/collection_20210624_194932.pkl', 'rb') as f:
    collection = pickle.load(f)

# I should completely remove the other bulletins
# for now this has to do...
bulletin_names = list(collection.bulletins.keys())
# target bulletins are those with usable related datasets and main points
target_bulletins = []
dictionary = dict()
for bulletin in bulletin_names:
    if len(collection.bulletins.get(bulletin).get('main-points')) and len(collection.bulletins.get(bulletin).get('related-datasets')) > 0:
        target_bulletins.append(bulletin)



with open('pickles/dataset_20210625_184837.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.head()

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...
3,businessindustryandtrade/business/businessserv...,date_and_percent,"Out of the 12 UK regions, 8 regions experience...",[/businessindustryandtrade/business/businessse...
4,businessindustryandtrade/business/businessserv...,date_and_percent,"West Midlands, Yorkshire and The Humber, Scotl...",[/businessindustryandtrade/business/businessse...


# RoBERTa

In [4]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
model = RobertaForMaskedLM.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [5]:
test_df = df[:3]

In [6]:
test_df

,bulletin,type,point,data
0,businessindustryandtrade/business/businessserv...,date_and_percent,"In 2019, approximate gross value added at basi...",[/businessindustryandtrade/business/businessse...
1,businessindustryandtrade/business/businessserv...,date_and_percent,"The non-financial services sector, which accou...",[/businessindustryandtrade/business/businessse...
2,businessindustryandtrade/business/businessserv...,date_and_percent,Total turnover and purchases of the UK non-fin...,[/businessindustryandtrade/business/businessse...


In [7]:
results, entity_set, entities = run_language_model(test_df, model, tokenizer, True)

100%|██████████| 3/3 [00:12<00:00,  4.02s/it]


In [8]:
results

[[(' 2019',
   0.535437285900116,
   '2019',
   0,
   'ccf75a2c157eacc5253f0e8a55b1ded9f5386d58'),
  (' £88 USD. USD billion',
   0.0005502063581090165,
   '£1,313.9 billion',
   0,
   'fc3346f2545ca5059bdb5150b998300c5001bea5'),
  (' £.bn.n',
   0.004800118851458449,
   '£42.8 billion',
   0,
   '81aa32270d547a368e7b806646c72069ba4fac43'),
  (' (aVA ',
   0.09505190574222439,
   '3.4%',
   0,
   'f308eb661ad1d6c8021cf9e22d8e8c7d9c5ff50e'),
  (' 2018',
   0.8733205199241638,
   '2018',
   0,
   '0a0bcf4b990f1926bc87fa662badadaf7e23d5fa')],
 [],
 [(' £,,,,, billion',
   0.49165046830007253,
   '£4,101.5 billion',
   2,
   '0ff2c435e1d192045540e12b44315fa61592a451'),
  (' £ in in billion in 2019 2020,',
   0.02682128286122674,
   '£2,761.4 billion',
   2,
   '1226559ad975ee3d495c004eb469d294d87bfd5d'),
  ('. sterling pounds billion sterling',
   0.06919503927726445,
   '£70.5 billion',
   2,
   '8444327426fe4840ba8daa1f5058bba3bee0eb42'),
  (' (.% respectively',
   0.15373660986963208,
 

- need to sort out how the answer_given / final_answer is sorted because it seems its wrong
- should think about all these repetitive predictions (respectively x3 etc)
- should fix that trailing whitespace before mask